In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
df_1=pd.read_csv('dc_part.csv')
df_2=pd.read_csv('chenqin5%.csv')
df_4=pd.read_csv('yunDataN.csv')
print(df_1.shape,df_2.shape,df_4.shape)

(124293, 147) (124293, 92) (124293, 74)


# Mergeing the 4 parts of the datasets with the zeor rate less than 95%

In [ ]:
data_parts=[df_1,df_2,df_4]
useful=[] # the column names for each parts that have less than 95% zeros
for ele in data_parts:
  useful_sub=[]
  for col in ele.columns:
    if ele[col].isin([0]).sum()/ele.shape[0]<0.95:
      useful_sub.append(col)
  useful.append(useful_sub)
useful[2]=useful[2][1:]  # drop unuseful column_name in the 4th part

In [ ]:
df_1=df_1.loc[:,useful[0]]
df_2=df_2.loc[:,useful[1]]
df_4=df_4.loc[:,useful[2]]

In [ ]:
result = pd.concat([df_1,df_2,df_4], axis=1, sort=False) # combine the dateset

In [ ]:
result.head()

,_key_client_id,_key_occurreddate_month,Total_incident_count,response_variable,incident_subcatgry_unknown_piv,prgcontct_other_organisation_piv,prgcontct_lwb_worker_piv,prgcontct_both_client_and_carer_piv,prgcontct_department_piv,prgcontct_family_piv,prgcontct_carer_piv,prgcontct_client_piv,prgcontct_meeting_group_of_people_piv_sum_n_days,prgcontct_other_organisation_piv_sum_n_days,prgcontct_educational_institution_piv_sum_n_days,prgcontct_lwb_worker_piv_sum_n_days,prgcontct_both_client_and_carer_piv_sum_n_days,prgcontct_health_professional_piv_sum_n_days,prgcontct_department_piv_sum_n_days,prgcontct_other_stakeholder_piv_sum_n_days,prgcontct_family_piv_sum_n_days,prgcontct_carer_piv_sum_n_days,prgcontct_client_piv_sum_n_days,prgcontct_meeting_group_of_people_piv_days_since,prgcontct_other_organisation_piv_days_since,prgcontct_meeting_lwb_internal_only_piv_days_since,prgcontct_educational_institution_piv_days_since,prgcontct_employer_piv_days_since,prgcontct_lwb_worker_piv_days_since,prgcontct_police_piv_days_since,prgcontct_both_client_and_carer_piv_days_since,prgcontct_health_professional_piv_days_since,prgcontct_department_piv_days_since,prgcontct_other_stakeholder_piv_days_since,prgcontct_family_piv_days_since,prgcontct_carer_piv_days_since,prgcontct_client_piv_days_since,progsubjcat_information_provided_piv,progsubjcat_case_discussion_planning_piv,progsubjcat_i_sight_client_incident_system_generated_piv,...,rowtype_reference_piv_avg_n_days,rowtype_client_address_addition_piv_days_since,rowtype_rp_end_piv_days_since,rowtype_rp_start_piv_days_since,rowtype_progress_note_piv_days_since,rowtype_diagnosis_piv_days_since,rowtype_medictn_start_piv_days_since,rowtype_client_plan_start_piv_days_since,rowtype_client_plan_end_piv_days_since,rowtype_keyworker_change_piv_days_since,rowtype_reference_piv_days_since,rowtype_medictn_end_piv_days_since,rowtype_plcment_end_piv_days_since,rowtype_incident_piv_days_since,rowtype_plcment_start_piv_days_since,rowtype_program_primary_service_type_start_piv_days_since,rowtype_client_alert_start_piv_days_since,rowtype_client_address_addition_piv_lag01,rowtype_rp_end_piv_lag01,rowtype_rp_start_piv_lag01,rowtype_progress_note_piv_lag01,rowtype_diagnosis_piv_lag01,rowtype_medictn_start_piv_lag01,rowtype_client_plan_start_piv_lag01,rowtype_client_plan_end_piv_lag01,rowtype_keyworker_change_piv_lag01,rowtype_reference_piv_lag01,rowtype_medictn_end_piv_lag01,rowtype_plcment_end_piv_lag01,rowtype_incident_piv_lag01,rowtype_plcment_start_piv_lag01,rowtype_program_primary_service_type_start_piv_lag01,rowtype_client_alert_start_piv_lag01,status_placement,status_client_plan,status_rp_mechanical_restraint,status_rp_restricted_access,status_rp_chemical_restraint,status_rp_other,Incident
0,0000D172-EA88-432F-8235-9FAA00D29072,2019-04-01T00:00:00.0,0,0,29,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,2868,3650,3650,0,1,0,...,1,603,355,376,890,515,492,761,555,345,2,425,469,703,527,577,488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,1,2,2,2,0
1,0000D172-EA88-432F-8235-9FAA00D29072,2019-05-01T00:00:00.0,0,0,31,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,22,3650,3650,0,0,0,...,1,603,355,376,21,515,492,761,555,345,0,425,469,703,527,577,488,1,1,1,2,1,1,1,1,1,38,1,1,1,1,1,1,2,2,1,2,2,2,0
2,0000D172-EA88-432F-8235-9FAA00D29072,2019-06-01T00:00:00.0,0,0,31,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,52,3650,3650,0,0,0,...,1,559,355,376,51,515,492,761,555,345,0,425,469,703,527,577,488,1,1,1,1,1,1,1,1,1,46,1,1,1,1,1,1,2,2,1,2,2,2,0
3,0000D172-EA88-432F-8235-9FAA00D29072,2019-07-01T00:00:00.0,0,0,32,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,3650,83,3650,483,0,1,0,...,1,21,355,376,20,515,492,761,555,345,0,425,469,703,527,577,488,3,1,1,1,1,1,1,1,1,40,1,1,1,1,1,1,2,2,1

### Parse 'Dates' and indexs it

In [ ]:
result.to_csv('Dataconcate.csv')

In [ ]:
Dataset=pd.read_csv('Dataconcate.csv',parse_dates=['_key_occurreddate_month'],index_col=['_key_client_id','_key_occurreddate_month'])

In [ ]:
Dataset.drop(['Unnamed: 0','response_variable'],axis=1).head()

Total_incident_count  ...  Incident
_key_client_id                       _key_occurreddate_month                        ...          
0000D172-EA88-432F-8235-9FAA00D29072 2019-04-01                                  0  ...         0
                                     2019-05-01                                  0  ...         0
                                     2019-06-01                                  0  ...         0
                                     2019-07-01                                  0  ...         0
                                     2019-08-01                                  0  ...         0

[5 rows x 150 columns]

In [ ]:
client_id=list(Dataset.index.get_level_values(0)) # the value of '_key_client_id' indexe

### Remove the Useless records

In [ ]:
from collections import Counter
occurrences = Counter(client_id)

In [ ]:
used_id=[]
usedless_id=[]
for key, value in occurrences.items():
  if value==12:  # number can be changed
    used_id.append(key)
  else:
    usedless_id.append(key)

In [ ]:
print(f'Useful client:  {len(used_id):d}\nUseless client: {len(usedless_id):d}')

Useful client:  7380
Useless client: 7007


In [ ]:
# remove the rows that the record of client is less than 12 month 
Dataset=Dataset.drop(usedless_id,level=0) 
Dataset.shape

(88560, 152)

In [ ]:
Dataset.drop(['Unnamed: 0','response_variable'],axis=1,inplace=True)

## Perparing Dateset for Modelling

### Spliting to Train and Test Set

In [ ]:
split=int(0.9*len(used_id))
train_index=used_id[:split] # the clients who will in trainset
test_index=used_id[split:] # the clients who will in testset

# Train dataset
train=Dataset.loc[train_index]
train_x = train.drop('Incident',axis=1)
train_y = train.Incident

# Test Dataset
test=Dataset.loc[test_index]
test_x = test.drop('Incident',axis=1)
test_y = test.Incident

print(train.shape,test.shape)

(79704, 150) (8856, 150)


### Normalize the Data

In [ ]:
from sklearn.preprocessing import RobustScaler
rs=RobustScaler()

train_x = rs.fit_transform(train_x)
test_x = rs.transform(test_x)

###Serializing the Dataset

In [ ]:
import numpy as np

def create_dataset(X, y, time_steps,window, number):
  xs, ys,channel = [],[],0
  while channel < number: # ensure all client data change to time series type

    for i in range(window - time_steps):
      shifft= window * channel
      v = X[i+shifft:(i + shifft + time_steps)]
      xs.append(v)
      ys.append(y[i + shifft + time_steps])

    channel+=1

  return np.array(xs),np.array(ys)

In [ ]:
TIME_STEPS = 4
WINDOW = 12
NUM_TRAIN = len(train_index)
NUM_TEST = len(test_index)

X_train, y_train = create_dataset(train_x,train_y, TIME_STEPS, WINDOW, NUM_TRAIN)
X_test, y_test = create_dataset(test_x, test_y, TIME_STEPS, WINDOW,NUM_TEST)

In [ ]:
# [n_samples, TIME_STEPS, n_features], [n_samples]
print(X_train.shape, y_train.shape)

(53136, 4, 149) (53136,)


## LSTM AutoEncoder

In [ ]:
def autoencoder_model(X):
    inputs = keras.Input(shape=(X.shape[1], X.shape[2]))
    L1 = layers.LSTM(16, activation='relu', return_sequences=True)(inputs)
    L2 = layers.LSTM(4, activation='relu', return_sequences=False)(L1)
    L4 = layers.RepeatVector(X.shape[1])(L2)
    L5 = layers.LSTM(4, activation='relu', return_sequences=True)(L4)
    L6 = layers.LSTM(16, activation='relu', return_sequences=True)(L5)
    output = layers.TimeDistributed(layers.Dense(X.shape[2]))(L6)    
    model = keras.Model(inputs=inputs, outputs=output)
    return model

In [ ]:
model = autoencoder_model(X_train)
model.compile(optimizer='adam', loss='mae')
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 4, 149)]          0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 4, 16)             10624     
_________________________________________________________________
lstm_24 (LSTM)               (None, 4)                 336       
_________________________________________________________________
repeat_vector_8 (RepeatVecto (None, 4, 4)              0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 4, 4)              144       
_________________________________________________________________
lstm_26 (LSTM)               (None, 4, 16)             1344      
_________________________________________________________________
time_distributed_6 (TimeDist (None, 4, 149)           

In [ ]:
model.compile(loss='mae',optimizer='adam')

In [ ]:
history = model.fit(
    X_train,y_train,
    epochs=10,
    batch_size=48,
    validation_split=0.1,shuffle=False)


Epoch 1/10


InvalidArgumentError: ignored